# Algorithm 6:  Handling Conditional Discrimination

In [459]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

In [460]:
df = pd.read_csv('../data/compas-scores-two-years.csv')
df.columns[df.isna().any()==False]

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree',
       'is_recid', 'is_violent_recid', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date', 'v_type_of_assessment',
       'v_decile_score', 'v_score_text', 'v_screening_date', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [461]:
selected_features = ['sex','age','race','priors_count','c_charge_degree','two_year_recid']

In [462]:
df_cleaned = df[selected_features][(df['race']=='Caucasian') | (df['race']=='African-American')].copy()
df_cleaned.head()
df_cleaned.index = np.arange(0,len(df_cleaned))
df_cleaned

,sex,age,race,priors_count,c_charge_degree,two_year_recid
0,Male,34,African-American,0,F,1
1,Male,24,African-American,4,F,1
2,Male,23,African-American,1,F,0
3,Male,41,Caucasian,14,F,1
4,Female,39,Caucasian,0,M,0
...,...,...,...,...,...,...
6145,Male,30,African-American,0,M,1
6146,Male,20,African-American,0,F,0
6147,Male,23,African-American,0,F,0
6148,Male,23,African-American,0,F,0


In [463]:
def Dexpl(df,expl,out='two_year_recid'):
    pdelta = []
    pstar = []
    for v in df[expl].unique():
        pa = 0 if np.isnan((df[df['race']=='African-American'][expl] == v).mean()) else (df[df['race']=='African-American'][expl] == v).mean()
        pc = 0 if np.isnan((df[df['race']=='Caucasian'][expl] == v).mean()) else (df[df['race']=='Caucasian'][expl] == v).mean()
        pdelta.append(pa-pc) 
        psa = (df[(df['race']=='African-American') & (df[expl] == v)][out] == 1).mean()
        psa = 0 if np.isnan(psa) else psa
        psc = (df[(df['race']=='Caucasian') & (df[expl] == v)][out] == 1).mean()
        psc = 0 if np.isnan(psc) else psc
        pstar.append((psa+psc)/2)
    return sum([pdelta[i]*ps for i,ps in enumerate(pstar)])

In [464]:
def delta(df_i,race):
    g_i = (df_i['race']==race).sum()
    p_star_a = (df_i[df_i['race']=='African-American']['two_year_recid'] == 1).mean()
    p_star_a = 0 if np.isnan(p_star_a) else p_star_a
    p_star_c = (df_i[df_i['race']=='Caucasian']['two_year_recid'] == 1).mean()
    p_star_c = 0 if np.isnan(p_star_c) else p_star_c
    p_star = (p_star_a+p_star_c)/2
    if race == 'African-American':
        return g_i*abs(p_star_a-p_star)
    if race == 'Caucasian':
        return g_i*abs(p_star_c-p_star)
    
def LocalMassaging(df,expl,numerical=['priors_count']):
    df_copy = df.copy()
    for attr in np.unique(df[expl]):
        X_df = df[df[expl]==attr]
        X = OneHotEncoder(drop='if_binary',sparse=False).fit_transform(X_df.drop(['race','two_year_recid',expl]+numerical,axis=1))
        
        X = np.hstack((X,X_df[numerical].to_numpy()))
        y = OneHotEncoder(drop='if_binary',sparse=False).fit_transform(df[df[expl]==attr][['two_year_recid']])
        np.random.seed(0)
        if not any(y):
            y[np.random.randint(low=0,high=len(y))]=1
        if all(y):
            y[np.random.randint(low=0,high=len(y))]=0
        if len(y) == 1:
            continue
        clf_logistic = LogisticRegression(random_state=0,max_iter=1000).fit(X, y.flatten())
        y_prob  = clf_logistic.predict_log_proba(X) 
        
        num_a = delta(X_df,'African-American')
        num_c= delta(X_df,'Caucasian')
        positive_index_sorted = np.argsort(y_prob[:,1])[np.sort(y_prob[:,1])>np.log(0.5)]
        negative_index_sorted = np.argsort(y_prob[:,1])[np.sort(y_prob[:,1])<np.log(0.5)]
        count_a,count_c = 0,0
        
        for index in positive_index_sorted:
            if df_copy.at[X_df.index[index],'race']== 'African-American':
                df_copy.at[X_df.index[index],'two_year_recid'] = 0
                count_a += 1
            if count_a >= num_a:
                break
        for index in reversed(negative_index_sorted):
            if df_copy.at[X_df.index[index],'race']== 'Caucasian':
                df_copy.at[X_df.index[index],'two_year_recid'] = 1
                count_c += 1
            if count_c>= num_c:
                break
        
    return df_copy
            
def LocalPreferentialSampling(df,expl,numerical=['priors_count']):
    df_copy = df.copy()
    index_to_drop_a,index_to_add_a=[],[]
    index_to_drop_c,index_to_add_c=[],[]
    for attr in np.unique(df[expl]):
        X_df = df[df[expl]==attr]
        X = OneHotEncoder(drop='if_binary',sparse=False).fit_transform(X_df.drop(['race','two_year_recid',expl]+numerical,axis=1))
        X = np.hstack((X,X_df[numerical].to_numpy()))
        y = OneHotEncoder(drop='if_binary',sparse=False).fit_transform(df[df[expl]==attr][['two_year_recid']])
        np.random.seed(0)
        if not any(y):
            y[np.random.randint(low=0,high=len(y))]=1
        if all(y):
            y[np.random.randint(low=0,high=len(y))]=0
        if len(y) == 1:
            continue
        clf_logistic = LogisticRegression(random_state=0,max_iter=1000).fit(X, y.flatten())
        y_prob  = clf_logistic.predict_log_proba(X) 
        
        num_a= delta(X_df,'African-American')
        num_c = delta(X_df,'Caucasian')
        positive_index_sorted = np.argsort(y_prob[:,1])[np.sort(y_prob[:,1])>np.log(0.5)]
        negative_index_sorted = np.argsort(y_prob[:,1])[np.sort(y_prob[:,1])<np.log(0.5)]
        count_drop_a,count_add_a,count_drop_c,count_add_c = 0,0,0,0    

        for index in positive_index_sorted:
            if (df_copy.at[X_df.index[index],'race'] == 'African-American') & (count_drop_a < 0.5*num_a):
                index_to_drop_a.append(X_df.index[index])
                count_drop_a += 1           
            if (df_copy.at[X_df.index[index],'race'] == 'Caucasian') & (count_add_c < 0.5*num_c):
                index_to_add_c.append(X_df.index[index])
                count_add_c += 1
            if (count_add_c>=0.5*num_c) & (count_drop_a>=0.5*num_a):
                break
                
                
        for index in reversed(negative_index_sorted):
            if (df_copy.at[X_df.index[index],'race'] == 'African-American') & (count_add_a < 0.5*num_a):
                index_to_add_a.append(X_df.index[index])
                count_add_a += 1           
            if (df_copy.at[X_df.index[index],'race'] == 'Caucasian') & (count_drop_c < 0.5*num_c):
                index_to_drop_c.append(X_df.index[index])
                count_drop_c += 1
            if (count_drop_c>=0.5*num_c) & (count_add_a>=0.5*num_a):
                break
        
    df_added = df.iloc[index_to_add_a+index_to_add_c]
    df_copy = df_copy.drop(index_to_drop_a+index_to_drop_c)   
    return pd.concat([df_copy,df_added])

def fpr(true,predict):
    return ((predict == 1) & (true == 0)).mean()

def fnr(true,predict):
    return ((predict == 0) & (true == 1)).mean()    

## Baseline

#### With `race`

In [465]:
##without race
numerical = ['priors_count']
# numerical = []
enc = OneHotEncoder(drop='if_binary',sparse=False)
X = enc.fit_transform(df_cleaned.drop(['two_year_recid']+numerical,axis=1))
X = np.hstack((X,df_cleaned[numerical].to_numpy()))
y = df_cleaned['two_year_recid'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [466]:
clf = LogisticRegression(random_state=0,max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [467]:
y_test_pred = clf.predict(X_test)
print('10-fold cross validation accuracy is: ',cross_val_score(clf, X_train, y_train, cv=10).mean())

10-fold cross validation accuracy is:  0.6739994725390651


In [468]:
aa_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='African-American'
ca_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='Caucasian'
aa_acc = (y_test_pred[aa_index] == y_test[aa_index]).mean()
ca_acc = (y_test_pred[ca_index] == y_test[ca_index]).mean()
print('Test accuracy is: ',clf.score(X_test,y_test))
print('Test accuracy in African-American group: ', aa_acc)
print('Test accuracy in Caucasian group: ',ca_acc)
print('Test accuracy difference in two groups: ', abs(aa_acc-ca_acc))
print('Test FPR: ',fpr(y_test,y_test_pred))
print('Test FPR in African-American group : ',fpr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FPR in Caucasian group: ',fpr(y_test[ca_index],y_test_pred[ca_index]))
print('Test FNR: ',fnr(y_test,y_test_pred))
print('Test FNR in African-American group: ',fnr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FNR in Caucasian group: ',fnr(y_test[ca_index],y_test_pred[ca_index]))

Test accuracy is:  0.6717226435536294
Test accuracy in African-American group:  0.6576576576576577
Test accuracy in Caucasian group:  0.6929347826086957
Test accuracy difference in two groups:  0.035277124951037964
Test FPR:  0.13759479956663057
Test FPR in African-American group :  0.16576576576576577
Test FPR in Caucasian group:  0.09510869565217392
Test FNR:  0.19068255687973998
Test FNR in African-American group:  0.17657657657657658
Test FNR in Caucasian group:  0.21195652173913043


#### Without `race`

In [469]:
enc.get_feature_names()

array(['x0_Male', 'x1_18', 'x1_19', 'x1_20', 'x1_21', 'x1_22', 'x1_23',
       'x1_24', 'x1_25', 'x1_26', 'x1_27', 'x1_28', 'x1_29', 'x1_30',
       'x1_31', 'x1_32', 'x1_33', 'x1_34', 'x1_35', 'x1_36', 'x1_37',
       'x1_38', 'x1_39', 'x1_40', 'x1_41', 'x1_42', 'x1_43', 'x1_44',
       'x1_45', 'x1_46', 'x1_47', 'x1_48', 'x1_49', 'x1_50', 'x1_51',
       'x1_52', 'x1_53', 'x1_54', 'x1_55', 'x1_56', 'x1_57', 'x1_58',
       'x1_59', 'x1_60', 'x1_61', 'x1_62', 'x1_63', 'x1_64', 'x1_65',
       'x1_66', 'x1_67', 'x1_68', 'x1_69', 'x1_70', 'x1_71', 'x1_72',
       'x1_73', 'x1_74', 'x1_75', 'x1_77', 'x1_78', 'x1_79', 'x1_80',
       'x1_83', 'x2_Caucasian', 'x3_M'], dtype=object)

In [470]:
clf = LogisticRegression(random_state=0,max_iter=1000)
X_train_no_race = X_train[:,[i for i in range(X_train.shape[1]) if i !=4]]
X_test_no_race = X_test[:,[i for i in range(X_test.shape[1]) if i !=4]]
clf.fit(X_train_no_race, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [471]:
y_test_pred = clf.predict(X_test_no_race)
print('10-fold cross validation accuracy is: ',cross_val_score(clf, X_train_no_race, y_train, cv=10).mean())

10-fold cross validation accuracy is:  0.6671157410459843


In [472]:
aa_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='African-American'
ca_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='Caucasian'
aa_acc = (y_test_pred[aa_index] == y_test[aa_index]).mean()
ca_acc = (y_test_pred[ca_index] == y_test[ca_index]).mean()
print('Test accuracy is: ',clf.score(X_test_no_race,y_test))
print('Test accuracy in African-American group: ', aa_acc)
print('Test accuracy in Caucasian group: ',ca_acc)
print('Test accuracy difference in two groups: ', abs(aa_acc-ca_acc))
print('Test FPR: ',fpr(y_test,y_test_pred))
print('Test FPR in African-American group : ',fpr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FPR in Caucasian group: ',fpr(y_test[ca_index],y_test_pred[ca_index]))
print('Test FNR: ',fnr(y_test,y_test_pred))
print('Test FNR in African-American group: ',fnr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FNR in Caucasian group: ',fnr(y_test[ca_index],y_test_pred[ca_index]))

Test accuracy is:  0.6663055254604551
Test accuracy in African-American group:  0.6558558558558558
Test accuracy in Caucasian group:  0.6820652173913043
Test accuracy difference in two groups:  0.026209361535448505
Test FPR:  0.13109425785482123
Test FPR in African-American group :  0.15855855855855855
Test FPR in Caucasian group:  0.08967391304347826
Test FNR:  0.20260021668472372
Test FNR in African-American group:  0.18558558558558558
Test FNR in Caucasian group:  0.22826086956521738


## Local Massaging

In [473]:
train, test = train_test_split(df_cleaned, test_size=0.15, random_state=0)
train.index = range(0,len(train))
test.index = range(0,len(test))
df_massaged = LocalMassaging(train,'age',numerical=['priors_count'])
numerical = ['priors_count']
enc = OneHotEncoder(drop='if_binary',sparse=False).fit(df_cleaned.drop(['two_year_recid']+numerical,axis=1))
X_train = enc.transform(df_massaged.drop(['two_year_recid']+numerical,axis=1))
X_train = np.hstack((X_train,df_massaged[numerical].to_numpy()))
y_train = df_massaged['two_year_recid'].to_numpy()

X_test = enc.transform(test.drop(['two_year_recid']+numerical,axis=1))
X_test = np.hstack((X_test,test[numerical].to_numpy()))
y_test = test['two_year_recid'].to_numpy()
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(5227, 67) (5227,) (923, 67) (923,)


In [474]:
from sklearn.svm import SVC
svc = SVC(random_state=0)
parameters = {'C':(0.5,0.75,1), 
              'kernel':('linear', 'poly', 'rbf', 'sigmoid')}
clf = GridSearchCV(svc, parameters,cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(random_state=0),
             param_grid={'C': (0.5, 0.75, 1),
                         'kernel': ('linear', 'poly', 'rbf', 'sigmoid')})

In [475]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [476]:
best_clf = SVC(random_state=0,C=0.5,kernel='linear')
best_clf.fit(X_train,y_train)
y_test_pred = best_clf.predict(X_test)
print('10-fold cross validation accuracy is: ',cross_val_score(best_clf, X_train_no_race, y_train, cv=10).mean())

10-fold cross validation accuracy is:  0.6483740284096321


In [477]:
aa_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='African-American'
ca_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='Caucasian'
aa_acc = (y_test_pred[aa_index] == y_test[aa_index]).mean()
ca_acc = (y_test_pred[ca_index] == y_test[ca_index]).mean()
print('Test accuracy is: ',best_clf.score(X_test,y_test))
print('Test accuracy in African-American group: ', aa_acc)
print('Test accuracy in Caucasian group: ',ca_acc)
print('Test accuracy difference in two groups: ', abs(aa_acc-ca_acc))
print('Test FPR: ',fpr(y_test,y_test_pred))
print('Test FPR in African-American group : ',fpr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FPR in Caucasian group: ',fpr(y_test[ca_index],y_test_pred[ca_index]))
print('Test FNR: ',fnr(y_test,y_test_pred))
print('Test FNR in African-American group: ',fnr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FNR in Caucasian group: ',fnr(y_test[ca_index],y_test_pred[ca_index]))

Test accuracy is:  0.6576381365113759
Test accuracy in African-American group:  0.6522522522522523
Test accuracy in Caucasian group:  0.6657608695652174
Test accuracy difference in two groups:  0.013508617312965154
Test FPR:  0.12892741061755147
Test FPR in African-American group :  0.14054054054054055
Test FPR in Caucasian group:  0.11141304347826086
Test FNR:  0.2134344528710726
Test FNR in African-American group:  0.2072072072072072
Test FNR in Caucasian group:  0.22282608695652173


## Local Preferential Sampling

In [478]:
train, test = train_test_split(df_cleaned, test_size=0.15, random_state=0)
train.index = range(0,len(train))
test.index = range(0,len(test))
df_massaged = LocalMassaging(train,'age',numerical=['priors_count'])
numerical = ['priors_count']
enc = OneHotEncoder(drop='if_binary',sparse=False).fit(df_cleaned.drop(['two_year_recid']+numerical,axis=1))
X_train = enc.transform(df_massaged.drop(['two_year_recid']+numerical,axis=1))
X_train = np.hstack((X_train,df_massaged[numerical].to_numpy()))
y_train = df_massaged['two_year_recid'].to_numpy()

X_test = enc.transform(test.drop(['two_year_recid']+numerical,axis=1))
X_test = np.hstack((X_test,test[numerical].to_numpy()))
y_test = test['two_year_recid'].to_numpy()
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(5227, 67) (5227,) (923, 67) (923,)


In [479]:
from sklearn.svm import SVC
svc = SVC(random_state=0)
parameters = {'C':(0.5,0.75,1), 
              'kernel':('linear', 'poly', 'rbf', 'sigmoid')}
clf = GridSearchCV(svc, parameters,cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(random_state=0),
             param_grid={'C': (0.5, 0.75, 1),
                         'kernel': ('linear', 'poly', 'rbf', 'sigmoid')})

In [480]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [481]:
best_clf = SVC(random_state=0,C=1,kernel='rbf')
best_clf.fit(X_train,y_train)
y_test_pred = best_clf.predict(X_test)
print('10-fold cross validation accuracy is: ',cross_val_score(best_clf, X_train, y_train, cv=10).mean())

10-fold cross validation accuracy is:  0.650666285722658


In [483]:
aa_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='African-American'
ca_index = enc.inverse_transform(X_test[:,:-1])[:,2]=='Caucasian'
aa_acc = (y_test_pred[aa_index] == y_test[aa_index]).mean()
ca_acc = (y_test_pred[ca_index] == y_test[ca_index]).mean()
print('Test accuracy is: ',best_clf.score(X_test,y_test))
print('Test accuracy in African-American group: ', aa_acc)
print('Test accuracy in Caucasian group: ',ca_acc)
print('Test accuracy difference in two groups: ', abs(aa_acc-ca_acc))
print('Test FPR: ',fpr(y_test,y_test_pred))
print('Test FPR in African-American group : ',fpr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FPR in Caucasian group: ',fpr(y_test[ca_index],y_test_pred[ca_index]))
print('Test FNR: ',fnr(y_test,y_test_pred))
print('Test FNR in African-American group: ',fnr(y_test[aa_index],y_test_pred[aa_index]))
print('Test FNR in Caucasian group: ',fnr(y_test[ca_index],y_test_pred[ca_index]))

Test accuracy is:  0.6489707475622969
Test accuracy in African-American group:  0.6342342342342342
Test accuracy in Caucasian group:  0.6711956521739131
Test accuracy difference in two groups:  0.03696141793967889
Test FPR:  0.12459371614301191
Test FPR in African-American group :  0.13513513513513514
Test FPR in Caucasian group:  0.10869565217391304
Test FNR:  0.22643553629469124
Test FNR in African-American group:  0.23063063063063063
Test FNR in Caucasian group:  0.22010869565217392
